# Test Notebook for Image Classification Project

In [2]:
# Imports
import tensorflow as tf
from PIL import Image
import numpy as np
import requests

In [3]:
# Get list of food categories
response = requests.get("https://world.openfoodfacts.org/categories.json")
categories = [cat['name'] for cat in response.json()['tags']]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
def load_model():
    # Load a pretrained deep learning model suitable for mobile applications
    model = tf.keras.applications.MobileNetV2(weights='imagenet')
    return model

def preprocess_image(image_path):
    # Use Keras API functions to preprocess the image to be classified
    img = tf.keras.utils.load_img(image_path, target_size=(224, 224), color_mode='rgb')
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

def predict_edible(image_path, model):
    # Get set of raw predictions, then determine whether top result is edible
    img = preprocess_image(image_path)
    predictions = model.predict(img)
    decoded = tf.keras.applications.mobilenet_v2.decode_predictions(predictions)[0]
    print(decoded)
    food_classes = ['apple', 'pizza']
    top_prediction = decoded[0][1] # Get class name for top result
    print(top_prediction)
    return top_prediction in food_classes

In [17]:
model = load_model()
predict_edible('C:/Users/mailr/code/edible-classifier/test-images/pizza.png', model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[('n07873807', 'pizza', np.float32(0.65991193)), ('n07747607', 'orange', np.float32(0.0632888)), ('n07768694', 'pomegranate', np.float32(0.04330279)), ('n03530642', 'honeycomb', np.float32(0.028241707)), ('n07745940', 'strawberry', np.float32(0.023253899))]
pizza


True